In [1]:
 # main

In [2]:
import sys
import os

# Projektordner app zum Suchpfad hinzufügen
sys.path.append(os.path.abspath("../"))

from main import main

In [3]:
main()

📄 Gefundene CSV-Dateien: 1
📄 Gefundene PDF-Dateien: 3
📂 Chroma-Verzeichnis gefunden. Vorhandene Daten werden geladen.
✅ Es wurden 34 Berufserfahrungen erkannt. Der Zeitraum erstreckt sich von January 2010 bis July 2025 (5662 Tage).
✅ Qualitätsprüfung bestanden.
⚠️ Keine neuen Dokumente hinzugefügt (alle bereits vorhanden?).
✅ PDF und CSV-Dateien erfolgreich verarbeitet und ins RAG eingefügt.
✅ Die Stellenauschreibung wurde erkannt.
✅ PDF-Inhalt wurde erfolgreich reduziert.
✅ HTML-Kurzprofil gespeichert unter: ../output/Kurzprofil_Marcel_Russ_job_posting_1.html
✅ Die Stellenauschreibung wurde erkannt.
✅ PDF-Inhalt wurde erfolgreich reduziert.
✅ HTML-Kurzprofil gespeichert unter: ../output/Kurzprofil_Marcel_Russ_job_posting_3.html
✅ Die Stellenauschreibung wurde erkannt.
✅ PDF-Inhalt wurde erfolgreich reduziert.
✅ HTML-Kurzprofil gespeichert unter: ../output/Kurzprofil_Marcel_Russ_job_posting_2.html
